In [ ]:
using Pkg
Pkg.activate("/home/bertschi/GitRepos/FinNetValu/")

In [ ]:
using FinNetValu
using Distributions
using LinearAlgebra
using SparseArrays
using ForwardDiff
using Plots
using DataFrames
using CSV

pyplot()

In [ ]:
r = 0.0
τ = 1.0
σ = [0.2, 0.2]
Lᵨ = cholesky([1.0 0; 0 1.0]).L

θ = BlackScholesParams(r, τ, σ, Lᵨ)

In [ ]:
function debtval(net, a₀, θ)
    N = numfirms(net)
    expectation(Z -> discount(θ) .* debtview(net, fixvalue(net, Aτ(a₀, θ, Z))),
                MonteCarloSampler(MvNormal(N, 1.0)),
                25000)
end

function insurancecost(net, a₀, θ)
    1 .- debtval(net, a₀, θ) ./ (discount(θ) .* net.d)
end

In [ ]:
function create_net(di₁, de₁, di₂, de₂)
    d = [di₁ + de₁, di₂ + de₂]
    Mᵈ = [0 (di₂ / d[2]); (di₁ / d[1]) 0]
    XOSModel(spzeros(2,2), Mᵈ, I, d)
end

function create_net_sym(di, de)
    d = [di + de, di + de]
    Mᵈ = [0 (di / d[2]); (di / d[1]) 0]
    XOSModel(spzeros(2,2), Mᵈ, I, d)
end

In [ ]:
di₂ = 0.6
de₂ = 0.4
df = DataFrame(di₁ = Float64[], de₁ = Float64[], in1 = Float64[], in2 = Float64[], a0 = Float64[])

for di₁ in range(0.2, length = 11, stop = 1)
    for de₁ in range(0.2, length = 11, stop = 1)
        for a₀ in range(0.2, length = 5, stop = 0.8)
            net = create_net(di₁, de₁, di₂, de₂)
            ins = insurancecost(net, a₀, θ)
            push!(df, (di₁, de₁, ins[1], ins[2], a₀))
       end
    end
end

df

In [ ]:
CSV.write("/tmp/foo.csv", df)

In [ ]:
? XOSModel

In [ ]:
di₂ = 0.6
de₂ = 0.4
df = DataFrame(di1 = Float64[], de1 = Float64[], A1 = Float64[], A2 = Float64[], sol1 = Bool[], sol2 = Bool[])

for di₁ in range(0.01, length = 5, stop = 0.8)
    for de₁ in range(0.01, length = 5, stop = 0.8)
        net = create_net(di₁, de₁, di₂, de₂)
        # net = create_net_sym(di₁, de₁)
        # for A1 in range(0.0, length = 51, stop = 3)
        for logA1 in range(-2, length = 51, stop = 1)
            for logA2 in range(-2, length = 51, stop = 1)
                A = 10 .^ [logA1, logA2]
                x = fixvalue(net, A)
                debt = debtview(net, x)
                sol  = solvent(net, x)
                ins = (net.d .- debt) ./ net.d
                push!(df, (di₁, de₁, A[1], A[2], sol[1], sol[2]))
            end
        end
    end
end

CSV.write("/tmp/baz.csv", df)

In [ ]:
di₂ = 0.6
de₂ = 0.4
df = DataFrame(di1 = Float64[], de1 = Float64[], in1 = Float64[], in2 = Float64[], lam1 = Float64[], lam2 = Float64[], sigma = Float64[], a0 = Float64[])

for di1 in range(0.01, length = 11, stop = 0.8)
    for de1 in range(0.01, length = 5, stop = 0.8)
        net = create_net(di1, de1, di₂, de₂)
        for sigma in [0.1, 0.2, 0.4]
            θ = BlackScholesParams(r, τ, sigma, Lᵨ)
            for a0 in [0.6, 0.8, 1, 1.2]
                N = numfirms(net)
                x = expectation(Z -> discount(θ) * fixvalue(net, Aτ(a0, θ, Z)),
                                MonteCarloSampler(MvNormal(N, 1.0)),
                                25000)
                v = equityview(net, x) .+ debtview(net, x)
                lam = equityview(net, x) ./ v
                ins = 1 .- debtview(net, x) ./ (discount(θ) .* net.d)
                push!(df, (di1, de1, ins[1], ins[2], lam[1], lam[2], sigma, a0))
            end
       end
    end
end

CSV.write("/tmp/qux.csv", df)

In [ ]:
using RCall

R"library(tidyverse)"
R"library(ggthemes)"
R"library(viridis)"

In [ ]:
R"read_csv(\"/tmp/baz.csv\") %>% ggplot(aes(A1, A2, fill = interaction(sol1, sol2))) + geom_tile() + scale_fill_colorblind() + theme_tufte() + facet_grid(de1 ~ di1) + scale_x_log10() + scale_y_log10() + geom_vline(xintercept = 1) + geom_hline(yintercept=1)"

In [ ]:
R"read_csv(\"/tmp/qux.csv\") %>% mutate(in12 = in1 + in2, lam12 = lam1 + lam2) %>% gather(key, val, matches(\"in\")) %>% ggplot(aes(di1, val, color = factor(sigma), linetype = factor(a0))) + geom_line() + facet_grid(de1 ~ key) + theme_tufte() + scale_color_colorblind()"